In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col, isnan, when, count, isnull
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.functions import *
import numpy as np

In [ ]:
spark = SparkSession.builder.appName("Apriori").getOrCreate()
df = spark.read.csv('D:\DAIICT\SEMESTER_2\BDP\dataset\Assignment-1_Data.csv', sep=';',header = True)
df.printSchema()

root
 |-- BillNo: string (nullable = true)
 |-- Itemname: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [ ]:
df.show()

+------+--------------------+--------+----------------+-----+----------+--------------+
|BillNo|            Itemname|Quantity|            Date|Price|CustomerID|       Country|
+------+--------------------+--------+----------------+-----+----------+--------------+
|536365|WHITE HANGING HEA...|       6|01.12.2010 08:26| 2,55|     17850|United Kingdom|
|536365| WHITE METAL LANTERN|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|CREAM CUPID HEART...|       8|01.12.2010 08:26| 2,75|     17850|United Kingdom|
|536365|KNITTED UNION FLA...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|RED WOOLLY HOTTIE...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|SET 7 BABUSHKA NE...|       2|01.12.2010 08:26| 7,65|     17850|United Kingdom|
|536365|GLASS STAR FROSTE...|       6|01.12.2010 08:26| 4,25|     17850|United Kingdom|
|536366|HAND WARMER UNION...|       6|01.12.2010 08:28| 1,85|     17850|United Kingdom|
|536366|HAND WARMER RED P...|   

In [ ]:
summary1 = df.summary().toPandas()
summary1

,summary,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,count,522064,520609,522064,522064,522064,388023,522064
1,mean,559950.7852856276,None,10.090435272303779,None,15.576812289966394,15316.931710233672,None
2,stddev,13452.750899837123,None,161.11052518229036,None,72.62169390055651,1721.8469642664234,None
3,min,536365,"""ASSORTED FLOWER COLOUR """"LEIS""""""",-1,01.02.2011 08:23,"-11062,06",12346,Australia
4,25%,547892.0,None,1.0,None,0.0,13950.0,None
5,50%,560601.0,None,3.0,None,0.0,15265.0,None
6,75%,571895.0,None,10.0,None,18.0,16837.0,None
7,max,A563187,wrongly sold sets,992,31.10.2011 17:19,"99,96",18287,Unspecified


Missing Values

In [ ]:
missing_counts = df.select([count(when(isnull(c) | isnan(c), c)).alias(c) for c in df.columns])
missing_counts.show()

+------+--------+--------+----+-----+----------+-------+
|BillNo|Itemname|Quantity|Date|Price|CustomerID|Country|
+------+--------+--------+----+-----+----------+-------+
|     0|    1455|       0|   0|    0|    134041|      0|
+------+--------+--------+----+-----+----------+-------+



Filling Null values in CustomerID with 999999

In [ ]:
df = df.fillna(999999,subset='CustomerID')

In [ ]:
summary2 = df.describe().toPandas()
summary2

,summary,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,count,522064,520609,522064,522064,522064,388023,522064
1,mean,559950.7852856276,None,10.090435272303779,None,15.576812289966394,15316.931710233672,None
2,stddev,13452.750899837,None,161.11052518229036,None,72.62169390055651,1721.8469642664231,None
3,min,536365,"""ASSORTED FLOWER COLOUR """"LEIS""""""",-1,01.02.2011 08:23,"-11062,06",12346,Australia
4,max,A563187,wrongly sold sets,992,31.10.2011 17:19,"99,96",18287,Unspecified


We can see that min Quantity is -1 which is not possible.

In [ ]:
df.filter(df.Quantity<=0).show()

+------+--------+--------+----------------+-----+----------+--------------+
|BillNo|Itemname|Quantity|            Date|Price|CustomerID|       Country|
+------+--------+--------+----------------+-----+----------+--------------+
|536589|    null|     -10|01.12.2010 16:50|    0|      null|United Kingdom|
|536764|    null|     -38|02.12.2010 14:42|    0|      null|United Kingdom|
|536996|    null|     -20|03.12.2010 15:30|    0|      null|United Kingdom|
|536997|    null|     -20|03.12.2010 15:30|    0|      null|United Kingdom|
|536998|    null|      -6|03.12.2010 15:30|    0|      null|United Kingdom|
|537000|    null|     -22|03.12.2010 15:32|    0|      null|United Kingdom|
|537001|    null|      -6|03.12.2010 15:33|    0|      null|United Kingdom|
|537003|    null|      -2|03.12.2010 15:33|    0|      null|United Kingdom|
|537004|    null|     -30|03.12.2010 15:34|    0|      null|United Kingdom|
|537005|    null|     -70|03.12.2010 15:35|    0|      null|United Kingdom|
|537006|    

Filtering out these rows as we can see it is noise in the data.

In [ ]:
df = df.filter(df.Quantity>0)
df.show()

+------+--------------------+--------+----------------+-----+----------+--------------+
|BillNo|            Itemname|Quantity|            Date|Price|CustomerID|       Country|
+------+--------------------+--------+----------------+-----+----------+--------------+
|536365|WHITE HANGING HEA...|       6|01.12.2010 08:26| 2,55|     17850|United Kingdom|
|536365| WHITE METAL LANTERN|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|CREAM CUPID HEART...|       8|01.12.2010 08:26| 2,75|     17850|United Kingdom|
|536365|KNITTED UNION FLA...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|RED WOOLLY HOTTIE...|       6|01.12.2010 08:26| 3,39|     17850|United Kingdom|
|536365|SET 7 BABUSHKA NE...|       2|01.12.2010 08:26| 7,65|     17850|United Kingdom|
|536365|GLASS STAR FROSTE...|       6|01.12.2010 08:26| 4,25|     17850|United Kingdom|
|536366|HAND WARMER UNION...|       6|01.12.2010 08:28| 1,85|     17850|United Kingdom|
|536366|HAND WARMER RED P...|   

Checking Null Values in Itemnames

In [ ]:
missing_counts = df.select([count(when(isnull('Itemname') | isnan('Itemname'), 'Itemname')).alias('Itemname')])
missing_counts.show()

+--------+
|Itemname|
+--------+
|     592|
+--------+



Removing rows where Itemnames is missing

In [ ]:
df = df.dropna()

In [ ]:
# NUll values
df.select([count(when(isnull(c) | isnan(c), c)).alias(c) for c in df.columns]).show()

+------+--------+--------+----+-----+----------+-------+
|BillNo|Itemname|Quantity|Date|Price|CustomerID|Country|
+------+--------+--------+----+-----+----------+-------+
|     0|       0|       0|   0|    0|         0|      0|
+------+--------+--------+----+-----+----------+-------+



In [ ]:
df = df.withColumn('Price', regexp_replace('Price', ',', '.'))
df = df.withColumn('Total_Price', round(df.Price * df.Quantity, 2))
df = df.withColumn('Hour', split('Date', ' ')[1])
df = df.withColumn('Date', split('Date', ' ')[0])
df = df.withColumn( 'Date', concat_ws('-', split('Date', '\.')[2], split('Date', '\.')[1], split('Date', '\.')[0]))
df = df.withColumn('Date', to_date('Date'))
df.show()

+------+--------------------+--------+----------+-----+----------+--------------+-----------+-----+
|BillNo|            Itemname|Quantity|      Date|Price|CustomerID|       Country|Total_Price| Hour|
+------+--------------------+--------+----------+-----+----------+--------------+-----------+-----+
|536365|WHITE HANGING HEA...|       6|2010-12-01| 2.55|     17850|United Kingdom|       15.3|08:26|
|536365| WHITE METAL LANTERN|       6|2010-12-01| 3.39|     17850|United Kingdom|      20.34|08:26|
|536365|CREAM CUPID HEART...|       8|2010-12-01| 2.75|     17850|United Kingdom|       22.0|08:26|
|536365|KNITTED UNION FLA...|       6|2010-12-01| 3.39|     17850|United Kingdom|      20.34|08:26|
|536365|RED WOOLLY HOTTIE...|       6|2010-12-01| 3.39|     17850|United Kingdom|      20.34|08:26|
|536365|SET 7 BABUSHKA NE...|       2|2010-12-01| 7.65|     17850|United Kingdom|       15.3|08:26|
|536365|GLASS STAR FROSTE...|       6|2010-12-01| 4.25|     17850|United Kingdom|       25.5|08:26|


Apriori Method

In [ ]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer, CountVectorizer
from pyspark.sql.types import IntegerType

In [ ]:
indexer = StringIndexer(inputCol = 'Itemname', outputCol = 'Item_Index')
data_indexed = indexer.fit(df).transform(df)
data_indexed.show()

+------+--------------------+--------+----------+-----+----------+--------------+-----------+-----+----------+
|BillNo|            Itemname|Quantity|      Date|Price|CustomerID|       Country|Total_Price| Hour|Item_Index|
+------+--------------------+--------+----------+-----+----------+--------------+-----------+-----+----------+
|536365|WHITE HANGING HEA...|       6|2010-12-01| 2.55|     17850|United Kingdom|       15.3|08:26|       0.0|
|536365| WHITE METAL LANTERN|       6|2010-12-01| 3.39|     17850|United Kingdom|      20.34|08:26|     437.0|
|536365|CREAM CUPID HEART...|       8|2010-12-01| 2.75|     17850|United Kingdom|       22.0|08:26|     448.0|
|536365|KNITTED UNION FLA...|       6|2010-12-01| 3.39|     17850|United Kingdom|      20.34|08:26|     267.0|
|536365|RED WOOLLY HOTTIE...|       6|2010-12-01| 3.39|     17850|United Kingdom|      20.34|08:26|     272.0|
|536365|SET 7 BABUSHKA NE...|       2|2010-12-01| 7.65|     17850|United Kingdom|       15.3|08:26|     324.0|
|

In [ ]:
df_group = data_indexed[['BillNo','Country','Itemname']].distinct()
df_group = df_group.groupBy('BillNo', 'Country').agg(collect_list('Itemname').alias('Basket'))
df_group.show()

+------+--------------+--------------------+
|BillNo|       Country|              Basket|
+------+--------------+--------------------+
|536394|United Kingdom|[FANCY FONT BIRTH...|
|536402|United Kingdom|[PAPER CHAIN KIT ...|
|536534|United Kingdom|[HAND WARMER SCOT...|
|536535|United Kingdom|[DOORMAT FANCY FO...|
|536575|United Kingdom|[LADS ONLY TISSUE...|
|536611|United Kingdom|[RED  HARMONICA I...|
|536628|United Kingdom|[CREAM CUPID HEAR...|
|536629|United Kingdom|[HAND WARMER UNIO...|
|536741|United Kingdom|[ASSORTED COLOUR ...|
|536771|United Kingdom|[JAM MAKING SET W...|
|536786|United Kingdom|[BAKING SET SPACE...|
|536806|United Kingdom|[CHRISTMAS TREE S...|
|536832|United Kingdom|[KEY CABINET MA C...|
|536838|United Kingdom|[FENG SHUI PILLAR...|
|536841|United Kingdom|[FIRST AID TIN, P...|
|536944|         Spain|[LUNCH BAG  BLACK...|
|536945|United Kingdom|[PLASTERS IN TIN ...|
|536972|United Kingdom|[RED KITCHEN SCAL...|
|536988|United Kingdom|[WHITE SKULL HOT ...|
|536992|Un

In [ ]:
countries = []
for country in df_group[['Country']].distinct().collect():
    countries.append(country['Country'])

In [ ]:
from pyspark.ml.fpm import FPGrowth

In [ ]:
minSupport=0.1
minConfidence=0.8
results = {}

In [ ]:
for country in countries:
    print(country)
    fpGrowth = FPGrowth(itemsCol="Basket", minSupport=minSupport, minConfidence=minConfidence)
    model = fpGrowth.fit(df_group.filter(df_group['Country']==country))
#     model.associationRules.show()
    results[country] = model.associationRules

Sweden


C:\Users\dhruv\anaconda3\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Germany
France
Belgium
Italy
Lithuania
Norway
Spain
Iceland
Switzerland
Japan
Poland
Portugal
Australia
Austria
United Kingdom
Netherlands
Singapore
Greece
Israel
Saudi Arabia
United Arab Emirates
Lebanon
Unspecified
USA
Brazil
Malta
Bahrain
RSA


In [ ]:
from pyspark.sql.functions import lit
country = countries[0]
apriori = results[country].withColumn('country', lit(country))

for country in countries[1:5]:
    df_temp = results[country].withColumn('country', lit(country))
    apriori =  apriori.union(df_temp)

In [ ]:
sqlCtx = SQLContext(spark)
apriori.createOrReplaceTempView("apriori")
rules = sqlCtx.sql("""SELECT antecedent, consequent, COUNT(DISTINCT country) as n_country , ROUND(AVG(lift), 3) as mean_lift, ROUND(MIN(lift), 3) as min_lift FROM apriori GROUP BY antecedent, consequent""")
rules.sort(col('n_country').desc(), col('mean_lift').desc()).show(30, truncate=False)

C:\Users\dhruv\anaconda3\lib\site-packages\pyspark\sql\context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----------------------------------------------------------------------+-------------------------------------+---------+---------+--------+
|antecedent                                                             |consequent                           |n_country|mean_lift|min_lift|
+-----------------------------------------------------------------------+-------------------------------------+---------+---------+--------+
|[REGENCY CAKESTAND 3 TIER]                                             |[POSTAGE]                            |4        |1.24     |1.021   |
|[RABBIT NIGHT LIGHT]                                                   |[POSTAGE]                            |3        |1.271    |1.021   |
|[RED TOADSTOOL LED NIGHT LIGHT]                                        |[POSTAGE]                            |3        |1.269    |1.021   |
|[ROUND SNACK BOXES SET OF4 WOODLAND]                                   |[POSTAGE]                            |3        |1.119    |1.021   |
|[PLASTERS IN

In [ ]:
from pyspark.ml.fpm import FPGrowth
# from time import time
minSupport=0.02
minConfidence=0.4

fpGrowth = FPGrowth(itemsCol="Basket", minSupport=minSupport, minConfidence=minConfidence)
model = fpGrowth.fit(df_group)
print('fitted')
results = model.associationRules
results.show()

fitted
+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[PAPER CHAIN KIT ...|[PAPER CHAIN KIT ...|0.46105919003115264|12.171828587988118|0.024445300886417443|
|[LUNCH BAG CARS B...|[LUNCH BAG  BLACK...|  0.439873417721519| 7.689529245501395|0.022958762319000164|
|[LUNCH BAG CARS B...|[LUNCH BAG SPACEB...|0.40717299578059074| 7.561843683397617| 0.02125199581566922|
|[LUNCH BAG CARS B...|[LUNCH BAG RED RE...|0.47362869198312235| 6.795037861366076| 0.02472058580630953|
|[LUNCH BAG CARS B...|[LUNCH BAG PINK P...| 0.4440928270042194| 8.825008771200917|0.023178990254913837|
|[LUNCH BAG CARS B...|[LUNCH BAG SUKI D...|0.40717299578059074| 8.290900361393351| 0.02125199581566922|
|[JUMBO STORAGE BA...|[JUMBO BAG RED RE...| 0.56119791666

In [ ]:
for column in ['confidence', 'lift', 'support']:
    results = results.withColumn(column, round(results[column], 3))

results.sort(col('lift').desc()).show(50, truncate=False)

+-----------------------------------+------------------------------------+------------------+------------------+--------------------+
|antecedent                         |consequent                          |confidence        |lift              |support             |
+-----------------------------------+------------------------------------+------------------+------------------+--------------------+
|[PINK REGENCY TEACUP AND SAUCER]   |[GREEN REGENCY TEACUP AND SAUCER]   |0.8210000000000052|22.84400000000026 |0.024000000000000205|
|[GREEN REGENCY TEACUP AND SAUCER]  |[PINK REGENCY TEACUP AND SAUCER]    |0.662             |22.844            |0.024               |
|[ROSES REGENCY TEACUP AND SAUCER]  |[GREEN REGENCY TEACUP AND SAUCER]   |0.689             |19.175            |0.028               |
|[GREEN REGENCY TEACUP AND SAUCER]  |[ROSES REGENCY TEACUP AND SAUCER]   |0.775             |19.175            |0.028               |
|[PINK REGENCY TEACUP AND SAUCER]   |[ROSES REGENCY TEACUP AND

In [ ]:
frequent_itemsets = model.freqItemsets
frequent_itemsets.show(100,truncate=False)

+---------------------------------------------------------------------+----+
|items                                                                |freq|
+---------------------------------------------------------------------+----+
|[PINK FAIRY CAKE CHILDRENS APRON]                                    |386 |
|[RECIPE BOX RETROSPOT]                                               |428 |
|[SET OF 3 BUTTERFLY COOKIE CUTTERS]                                  |485 |
|[PAPER CHAIN KIT EMPIRE]                                             |372 |
|[JUMBO  BAG BAROQUE BLACK WHITE]                                     |641 |
|[WOODLAND CHARLOTTE BAG]                                             |524 |
|[SET OF 20 KIDS COOKIE CUTTERS]                                      |405 |
|[DOORMAT HEARTS]                                                     |465 |
|[POTTERING IN THE SHED METAL SIGN]                                   |367 |
|[HAND OVER THE CHOCOLATE   SIGN]                                     |589 |

In [ ]:
fre_df = frequent_itemsets.filter(size('items') >= 2)
fre_df.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[JUMBO STORAGE BA...| 431|
|[LUNCH BAG CARS B...| 417|
|[LUNCH BAG CARS B...| 386|
|[LUNCH BAG CARS B...| 449|
|[LUNCH BAG WOODLA...| 402|
|[LUNCH BAG WOODLA...| 427|
|[GARDENERS KNEELI...| 461|
|[ROSES REGENCY TE...| 387|
|[LUNCH BAG PINK P...| 455|
|[LUNCH BAG PINK P...| 510|
|[LUNCH BAG PINK P...| 421|
|[LUNCH BAG SUKI D...| 406|
|[LUNCH BAG SUKI D...| 376|
|[LUNCH BAG SUKI D...| 448|
|[LUNCH BAG SUKI D...| 386|
|[LUNCH BAG RED RE...| 419|
|[RED HANGING HEAR...| 439|
|[GREEN REGENCY TE...| 506|
|[WOODEN FRAME ANT...| 466|
|[JUMBO BAG PINK P...| 540|
+--------------------+----+
only showing top 20 rows

